In [2]:
data = open(
    '/content/drive/MyDrive/Colab Notebooks/AOC/2022/day11.txt',
    'r').read().split("\n\n")

In [3]:
import typing
import math 

class Monkey:
    def __init__(self, tag: int):
        self.tag = tag
        self.items = []
        self.op = None
        self.test = None
        self.throwsTo = []  #[True, False]
        self.numInspected = 0

    def parse_op(self, operation: str) -> typing.Callable[[int], int]:
        func = 'lambda x'
        op, y = '', 'y'
        rule = operation.rstrip().split("=")[1]
        for char in rule.lstrip().split(" ")[1:]:
            if char == "old": y = 'x'
            elif char.isnumeric(): y = int(char)
            else: op = char
                
        func += f": x {op} {y}"
        self.op = eval(func)
        return eval(func)

    def add_items(self, items: str) -> None:
        items = list(map(int, items.split(", ")))
        self.items += items
    
    def inspect(self, monkeys) -> None:
        for old in self.items.copy():
            # print(f"Monkey {self.tag} inspecting item of worry level {old}")
            self.numInspected += 1
            new = self.op(old)
            # print(f"New worry level: {new}")
            current = math.floor(new/3)
            # print(f"After inspection, current worry level {current}")
            if current % self.test == 0:
                # print(f"Current worry level divisible by {self.test}, throwing to Monkey {self.throwsTo[0]}.")
                monkeys[self.throwsTo[0]].items.append(current)
            else:
                # print(f"Current worry level not divisible by {self.test}, throwing to Monkey {self.throwsTo[1]}.")
                monkeys[self.throwsTo[1]].items.append(current)
            self.items.remove(old)

    def inspectP2(self, monkeys) -> None:
        '''To manage worry levels, we can use this fact on modulo arithmetic: 
        Take A, B as non-negative integers then (A + B) % n = ((A % n) + (B % n)) % n
        
        Since the test involves checking for divisibility on the same 7 factors,
        the product (call x) is the smallest number divisible by all factors.
        We mod the worry level by x, wrapping worry levels in [0, x-1]. 

        If original worry level > x, i.e. worry level = x + D, D > 0 
        then D = worry % x is divisible by n => x + D is divisible by n.
        ''' 
        for old in self.items.copy():
            self.numInspected += 1
            new = self.op(old)
            current = new % 9699690
            if current % self.test == 0:
                monkeys[self.throwsTo[0]].items.append(current)
            else:
                monkeys[self.throwsTo[1]].items.append(current)
            self.items.remove(old)

    def __repr__(self) -> str:
        print(f"Monkey {self.tag}")


In [4]:
def parser(input):
    '''Parse monkey inputs
    '''
    monkas = []
    for idx, monkey_bio in enumerate(input):
        monkey = Monkey(idx)
        for detail in monkey_bio.splitlines()[1:]:
            arg = detail.lstrip().split(": ")
            # print(arg)
            if arg[0] == "Starting items": monkey.add_items(arg[1])
            elif arg[0] == "Operation": monkey.parse_op(arg[1])
            elif arg[0] == "Test": monkey.test = int(arg[1].split(" ")[-1])
            else: monkey.throwsTo.append(int(arg[1].split(" ")[-1]))

        monkas.append(monkey)
    return monkas

In [5]:
from functools import reduce
def findLevelOfMonkeyBiz(Monkeys, rounds, part2=False):
    round = 0
    inspections = []
    while round < rounds:
        for monkey in Monkeys:
            if part2: monkey.inspectP2(Monkeys)
            else: monkey.inspect(Monkeys)
        round += 1
        # print(f"After {round} rounds, items in each monkey:")
        # for monkey in Monkeys:
        #     print(f"Monkey {monkey.tag}: {monkey.items}")
    for monkey in Monkeys:
        inspections.append(monkey.numInspected)
    
    inspections.sort()
    return reduce(lambda x, y: x*y, inspections[-2:])

In [6]:
Monkeys = parser(data)
findLevelOfMonkeyBiz(Monkeys, 20)

61503

In [7]:
Monkeys = parser(data)
findLevelOfMonkeyBiz(Monkeys, 10000, True)

14081365540